<a href="https://colab.research.google.com/github/aksy1999/NLP_Assignment_2/blob/master/Amit_16110011_Assignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
#16110011 -Amit Kumar Singh Yadav, 
#TASK 2: NEURAL APPROACH LANGUAGE MODELLING | COllaborators - TA's (USE CODE GIVEN IN DEEP LEARNING CODING SESSION)
#Definng all headers 
import numpy as np
import tensorflow as tf
import re
import random
import collections
import time
from tensorflow import keras
from tensorflow.contrib import rnn
from __future__ import print_function
from operator import itemgetter
from nltk import word_tokenize, sent_tokenize
from string import punctuation
from nltk.util import ngrams
from sklearn.model_selection import train_test_split 
import nltk
nltk.download('punkt')
import warnings
warnings.filterwarnings("ignore")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [9]:
# Pre-Processing Dataset -Filtering out symbols, newlines. 
#Parsing and dividing in 80-20 ratio

#CODE SIMILAR TO PROCESSING DONE IN 1st part using Language Model
# DATASET- Jane Austen Novels: The Complete Works of Jane Austen 
#take one line, neglect empty line and then add it to line_set then lower down whole set(string)
with open("31100.txt", 'rt') as source_file:  
    line_set = ['']
    for line in (l.rstrip() for l in source_file):
        if line != '' or line_set[-1] !='\n':                 
            line_set.append(line + '\n')
    text = "".join(line_set)
text = text.lower()  

# Tokenizing sentences from processed text
vocabulary = {}#initilaizing vocab dict
corpus = []#intializing corpus
sentence_set = sent_tokenize(text)

for sentence in sentence_set: #taking one sentence from sentence list
    words = word_tokenize(sentence) #work tokenizing each word of that sentence
    processed_sentence = ['<s>'] #making processed sentence out of it and adding START <s> symbol 
    for word in words:
        if len(set(word).intersection(punctuation)) == 0 and (len(word)>1 or word == 'a' or word == 'i') :
            processed_sentence.append(word)#adding processed and selected word to my new processed sentence
            if word in vocabulary.keys():#adding in vobaulary list if not exist else increasing frequecny by 1 if exists
                vocabulary[word] += 1
            else:
                vocabulary[word] = 1
                
    processed_sentence.append('</s>')#adding END </s> to the end of my sentence
    corpus.append(" ".join(processed_sentence))#adding processed sentence to the corpus

#Now Processed Corpus is ready    
number_of_sentence = len(corpus)
# Vocabulary list doesn't contain the START and END word used in Pre-Processing
vocabulary['<s>'] = 2*number_of_sentence
vocabulary['</s>'] = 2*number_of_sentence
# Now we have Processed sentence corpus and vocabulary list. 
#Processed Corpus DATA ANALYSIS  
tokens = sum(vocabulary.values())
types = len(vocabulary)
print ('Number of Sentences = ' + str(number_of_sentence))
print ('Number of Tokens = ' + str(tokens))
print ('Number of Types = ' + str(types))
print ("TTR= " + str(types/tokens))
# train-test split in 80:20 ratio
train_data, test_data = train_test_split(corpus, test_size=0.20, random_state=42)

Number of Sentences = 35056
Number of Tokens = 914604
Number of Types = 14651
TTR= 0.01601895465141198


In [10]:
#defining function for calculting elapsed Time, read data and bild dataset- MODIFIED THE CODE GIVEN IN CODING SESSION 

def read_data(content):
    content = [x.strip() for x in content]
    content = [word for i in range(len(content)) for word in content[i].split()]
    content = np.array(content)
    return content

def elapsed(sec):
    if sec<60:
        return str(sec) + " sec"
    elif sec<(60*60):
        return str(sec/60) + " min"
    else:
        return str(sec/(60*60)) + " hr"

def build_dataset(words):
    count = collections.Counter(words).most_common()
    dictionary = dict()
    for word, _ in count:
        dictionary[word] = len(dictionary)
    reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return dictionary, reverse_dictionary

# Importing Train Data and defining build dataset function - USED CODE GIVEN IN CODING SESSION
start_time = time.time()
training_data = read_data(train_data)
testing_data = read_data(test_data)
print("Loaded training data...")
# CREATING DICTIONARY USING build_dataset function
dictionary, reverse_dictionary = build_dataset(training_data)
vocab_size = len(dictionary)


Loaded training data...


In [0]:
#Defining the train_text and decoding function- as done in coding session
# A dictionary mapping words to an integer index
word_index = dictionary.copy()
# The first indices are reserved
word_index = {k:(v+3) for k,v in word_index.items()}
word_index["<PAD>"] = 0
word_index["<UNK>"] = 1  # unknown
word_index["<UNUSED>"] = 2

reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])
train_text = []
for i in train_data:
  train_text.append(decode_review(i))

In [12]:
#CODE FOR TASK-2a Vanilla RNN BASED LANGUAGE GENERATION (QUADGRAMS)
tf.reset_default_graph()

# Parameters
learning_rate = 0.0001
training_iters = 40000
display_step = 1000
#CHANGE VALUE OF n_gram for getting N-GRAM MODEL
n_gram = 4


logs_path = '/tmp/tensorflow/rnn_words'
writer = tf.summary.FileWriter(logs_path)

# number of units in RNN cell
n_hidden = 128

# tf Graph input
x = tf.placeholder("float", [None, n_gram, 1])
y = tf.placeholder("float", [None, vocab_size])

# RNN output node weights and biases
weights = {
    'out': tf.Variable(tf.random_normal([n_hidden, vocab_size]))
}
biases = {
    'out': tf.Variable(tf.random_normal([vocab_size]))
}

def RNN(x, weights, biases):

    x = tf.reshape(x, [-1, n_gram])
    x = tf.split(x,n_gram,1)
    rnn_cell = rnn.BasicRNNCell(n_hidden)
    outputs, states = rnn.static_rnn(rnn_cell, x, dtype=tf.float32)
    return tf.matmul(outputs[-1], weights['out']) + biases['out']

pred = RNN(x, weights, biases)

# Loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.RMSPropOptimizer(learning_rate=learning_rate).minimize(cost)

# Model evaluation
correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initializing the variables
init = tf.global_variables_initializer()

# Launch the graph
with tf.Session() as session:
    session.run(init)
    step = 0
    offset = random.randint(0,n_gram+1)
    end_offset = n_gram + 1
    acc_total = 0
    loss_total = 0
    loss_overall = 0
    writer.add_graph(session.graph)

    while step < training_iters:
        # Generate a minibatch. Add some randomness on selection process.
        if offset > (len(training_data)-end_offset):
            offset = random.randint(0, n_gram+1)

        symbols_in_keys = [ [dictionary[ str(training_data[i])]] for i in range(offset, offset+n_gram) ]
        symbols_in_keys = list(np.reshape(np.array(symbols_in_keys), [-1, n_gram, 1]))

        symbols_out_onehot = np.zeros([vocab_size], dtype=float)
        symbols_out_onehot[dictionary[str(training_data[offset+n_gram])]] = 1.0
        symbols_out_onehot = np.reshape(symbols_out_onehot,[1,-1])

        _, acc, loss, onehot_pred = session.run([optimizer, accuracy, cost, pred], \
                                                feed_dict={x: symbols_in_keys, y: symbols_out_onehot})
        loss_total += loss
        loss_overall += loss
        acc_total += acc
        if (step+1) % display_step == 0:
            print("Iteration= " + str(step+1) + ", Average Loss= " + \
                  "{:.6f}".format(loss_total/display_step) + ", Average Accuracy= " + \
                  "{:.2f}%".format(100*acc_total/display_step))
            acc_total = 0
            loss_total = 0
            symbols_in = [training_data[i] for i in range(offset, offset + n_gram)]
            symbols_out = training_data[offset + n_gram]
            symbols_out_pred = reverse_dictionary[int(tf.argmax(onehot_pred, 1).eval())]
        step += 1
        offset += (n_gram+1)
    print("Optimization Finished!")
    print("Elapsed time: ", elapsed(time.time() - start_time))
    
    num_start = 0
    words = [word_index['<PAD>']] * (n_gram-1) + ['<s>']
    overall = list(words)
    sentence = ''
    num_words = 0
    while num_start < n_gram and num_words < 500:
        try:
            symbols_in_keys = [dictionary[str(words[i])] for i in range(len(words))]
            keys = np.reshape(np.array(symbols_in_keys), [-1, n_gram, 1])
            onehot_pred = session.run(pred, feed_dict={x: keys})
            onehot_pred_index = int(tf.argmax(onehot_pred, 1).eval())
            if onehot_pred_index == dictionary['</s>']:
                sentence += ". "
                num_start += 1
            else:
                sentence = "%s %s" % (sentence,reverse_dictionary[onehot_pred_index])
            symbols_in_keys = list(symbols_in_keys[1:])
            symbols_in_keys.append(onehot_pred_index)
        except:
            onehot_pred_index = dictionary['the']
            sentence = "%s %s" % (sentence,reverse_dictionary[onehot_pred_index])
            symbols_in_keys = list(symbols_in_keys[1:])
            symbols_in_keys.append(onehot_pred_index)
        num_words += 1
        

    print(sentence)

Instructions for updating:
This class is equivalent as tf.keras.layers.SimpleRNNCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell, unroll=True)`, which is equivalent to this API
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Iteration= 1000, Average Loss= 31.952727, Average Accuracy= 1.20%
Iteration= 2000, Average Loss= 24.466701, Average Accuracy= 2.10%
Iteration= 3000, Average Loss= 19.005330, Average Accuracy= 3.10%
Iteration= 400

In [13]:
#TASK_2 FINDING PERPLEXITY 
perplexity = tf.exp(loss)
with tf.Session() as session:
  print("Perplexity for RNN Based Quadram MOdel= "+str(session.run(perplexity)))

Perplexity for RNN Based Quadram MOdel= 64720.492


In [14]:
# SAME CODE JUST CHANGED THE MODEL TO LSTM BASED
tf.reset_default_graph()
# tf Graph input
x = tf.placeholder("float", [None, n_gram, 1])
y = tf.placeholder("float", [None, vocab_size])

# RNN output node weights and biases
weights = {
    'out': tf.Variable(tf.random_normal([n_hidden, vocab_size]))
}
biases = {
    'out': tf.Variable(tf.random_normal([vocab_size]))
}

def RNN(x, weights, biases):

    x = tf.reshape(x, [-1, n_gram])

    # Generate a n_input-element sequence of inputs
    x = tf.split(x,n_gram,1)

    # 1-layer LSTM with n_hidden units 
    rnn_cell = rnn.LSTMCell(n_hidden, reuse =tf.AUTO_REUSE)

    outputs, states = rnn.static_rnn(rnn_cell, x, dtype=tf.float32)
    # there are n_input outputs but we only want the last output
    return tf.matmul(outputs[-1], weights['out']) + biases['out']

pred = RNN(x, weights, biases)

# Loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.RMSPropOptimizer(learning_rate=learning_rate).minimize(cost)

# Model evaluation
correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initializing the variables
init = tf.global_variables_initializer()

# Launch the graph
with tf.Session() as session:
    session.run(init)
    step = 0
    offset = random.randint(0,n_gram+1)
    end_offset = n_gram + 1
    acc_total = 0
    loss_total = 0
    loss_overall = 0
    writer.add_graph(session.graph)

    while step < training_iters:
        # Generate a minibatch. Add some randomness on selection process.
        if offset > (len(training_data)-end_offset):
            offset = random.randint(0, n_gram+1)

        symbols_in_keys = [ [dictionary[ str(training_data[i])]] for i in range(offset, offset+n_gram) ]
        symbols_in_keys = list(np.reshape(np.array(symbols_in_keys), [-1, n_gram, 1]))

        symbols_out_onehot = np.zeros([vocab_size], dtype=float)
        symbols_out_onehot[dictionary[str(training_data[offset+n_gram])]] = 1.0
        symbols_out_onehot = np.reshape(symbols_out_onehot,[1,-1])

        _, acc, loss, onehot_pred = session.run([optimizer, accuracy, cost, pred], \
                                                feed_dict={x: symbols_in_keys, y: symbols_out_onehot})
        loss_total += loss
        loss_overall += loss
        acc_total += acc
        if (step+1) % display_step == 0:
            print("Iter= " + str(step+1) + ", Average Loss= " + \
                  "{:.6f}".format(loss_total/display_step) + ", Average Accuracy= " + \
                  "{:.2f}%".format(100*acc_total/display_step))
            acc_total = 0
            loss_total = 0
            symbols_in = [training_data[i] for i in range(offset, offset + n_gram)]
            symbols_out = training_data[offset + n_gram]
            symbols_out_pred = reverse_dictionary[int(tf.argmax(onehot_pred, 1).eval())]
            
        step += 1
        offset += (n_gram+1)
    print("Optimization Finished!")
    print("Elapsed time: ", elapsed(time.time() - start_time))
    
    num_start = 0
    words = [word_index['<PAD>']] * (n_gram-1) + ['<s>']
    overall = list(words)
    sentence = ''
    num_words = 0
    while num_start < 5 and num_words < 500:
        try:
            symbols_in_keys = [dictionary[str(words[i])] for i in range(len(words))]
            keys = np.reshape(np.array(symbols_in_keys), [-1, n_gram, 1])
            onehot_pred = session.run(pred, feed_dict={x: keys})
            onehot_pred_index = int(tf.argmax(onehot_pred, 1).eval())
            if onehot_pred_index == dictionary['</s>']:
                sentence += ". "
                num_start += 1
            else:
                sentence = "%s %s" % (sentence,reverse_dictionary[onehot_pred_index])
            symbols_in_keys = list(symbols_in_keys[1:])
            symbols_in_keys.append(onehot_pred_index)
        except:
            onehot_pred_index = dictionary['the']
            sentence = "%s %s" % (sentence,reverse_dictionary[onehot_pred_index])
            symbols_in_keys = list(symbols_in_keys[1:])
            symbols_in_keys.append(onehot_pred_index)
        num_words += 1
        

    print(sentence)

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Iter= 1000, Average Loss= 13.878376, Average Accuracy= 0.00%
Iter= 2000, Average Loss= 10.719683, Average Accuracy= 0.40%
Iter= 3000, Average Loss= 9.785787, Average Accuracy= 3.50%
Iter= 4000, Average Loss= 9.282362, Average Accuracy= 5.50%
Iter= 5000, Average Loss= 8.891389, Average Accuracy= 6.00%
Iter= 6000, Average Loss= 8.429275, Average Accuracy= 7.60%
Iter= 7000, Average Loss= 8.226546, Average Accuracy= 7.90%
Iter= 8000, Average Loss= 8.079347, Average Accuracy= 8.50%
Iter= 9000, Average Loss= 7.642736, Average Accuracy= 7.70%
Iter= 10000, Average Loss= 7.690950, Average Accuracy= 7.40%
Iter= 11000, Average Loss= 7.433869, Average Accuracy= 8.30%
Iter= 12000, Average Loss= 7.619117, Average Accuracy= 7.50%
Iter= 13000, Average Loss= 7.392291, Average Accuracy= 7.40%
Iter= 14000, Average Loss= 7.406154, Average Accuracy= 7.60%
Iter= 15000, Average Los

In [15]:

perplexity_LSTM =tf.exp(loss)
with tf.Session() as session:
  print("Perplexity for QUADRAM LSTM BASED model= "+ str(session.run(perplexity_LSTM)))

Perplexity for QUADRAM LSTM BASED model= 1795.1171


In [16]:
#CODE FOR TASK-2a Vanilla RNN BASED LANGUAGE GENERATION (TRIGRAMS)
tf.reset_default_graph()

# Parameters
learning_rate = 0.0001
training_iters = 40000
display_step = 1000
#CHANGE VALUE OF n_gram for getting N-GRAM MODEL
n_gram = 3


logs_path = '/tmp/tensorflow/rnn_words'
writer = tf.summary.FileWriter(logs_path)

# number of units in RNN cell
n_hidden = 128

# tf Graph input
x = tf.placeholder("float", [None, n_gram, 1])
y = tf.placeholder("float", [None, vocab_size])

# RNN output node weights and biases
weights = {
    'out': tf.Variable(tf.random_normal([n_hidden, vocab_size]))
}
biases = {
    'out': tf.Variable(tf.random_normal([vocab_size]))
}

def RNN(x, weights, biases):

    x = tf.reshape(x, [-1, n_gram])
    x = tf.split(x,n_gram,1)
    rnn_cell = rnn.BasicRNNCell(n_hidden)
    outputs, states = rnn.static_rnn(rnn_cell, x, dtype=tf.float32)
    return tf.matmul(outputs[-1], weights['out']) + biases['out']

pred = RNN(x, weights, biases)

# Loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.RMSPropOptimizer(learning_rate=learning_rate).minimize(cost)

# Model evaluation
correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initializing the variables
init = tf.global_variables_initializer()

# Launch the graph
with tf.Session() as session:
    session.run(init)
    step = 0
    offset = random.randint(0,n_gram+1)
    end_offset = n_gram + 1
    acc_total = 0
    loss_total = 0
    loss_overall = 0
    writer.add_graph(session.graph)

    while step < training_iters:
        # Generate a minibatch. Add some randomness on selection process.
        if offset > (len(training_data)-end_offset):
            offset = random.randint(0, n_gram+1)

        symbols_in_keys = [ [dictionary[ str(training_data[i])]] for i in range(offset, offset+n_gram) ]
        symbols_in_keys = list(np.reshape(np.array(symbols_in_keys), [-1, n_gram, 1]))

        symbols_out_onehot = np.zeros([vocab_size], dtype=float)
        symbols_out_onehot[dictionary[str(training_data[offset+n_gram])]] = 1.0
        symbols_out_onehot = np.reshape(symbols_out_onehot,[1,-1])

        _, acc, loss, onehot_pred = session.run([optimizer, accuracy, cost, pred], \
                                                feed_dict={x: symbols_in_keys, y: symbols_out_onehot})
        loss_total += loss
        loss_overall += loss
        acc_total += acc
        if (step+1) % display_step == 0:
            print("Iteration= " + str(step+1) + ", Average Loss= " + \
                  "{:.6f}".format(loss_total/display_step) + ", Average Accuracy= " + \
                  "{:.2f}%".format(100*acc_total/display_step))
            acc_total = 0
            loss_total = 0
            symbols_in = [training_data[i] for i in range(offset, offset + n_gram)]
            symbols_out = training_data[offset + n_gram]
            symbols_out_pred = reverse_dictionary[int(tf.argmax(onehot_pred, 1).eval())]
        step += 1
        offset += (n_gram+1)
    print("Optimization Finished!")
    print("Elapsed time: ", elapsed(time.time() - start_time))
    
    num_start = 0
    words = [word_index['<PAD>']] * (n_gram-1) + ['<s>']
    overall = list(words)
    sentence = ''
    num_words = 0
    while num_start < n_gram and num_words < 500:
        try:
            symbols_in_keys = [dictionary[str(words[i])] for i in range(len(words))]
            keys = np.reshape(np.array(symbols_in_keys), [-1, n_gram, 1])
            onehot_pred = session.run(pred, feed_dict={x: keys})
            onehot_pred_index = int(tf.argmax(onehot_pred, 1).eval())
            if onehot_pred_index == dictionary['</s>']:
                sentence += ". "
                num_start += 1
            else:
                sentence = "%s %s" % (sentence,reverse_dictionary[onehot_pred_index])
            symbols_in_keys = list(symbols_in_keys[1:])
            symbols_in_keys.append(onehot_pred_index)
        except:
            onehot_pred_index = dictionary['the']
            sentence = "%s %s" % (sentence,reverse_dictionary[onehot_pred_index])
            symbols_in_keys = list(symbols_in_keys[1:])
            symbols_in_keys.append(onehot_pred_index)
        num_words += 1
        

    print(sentence)
    

#TASK_2 FINDING PERPLEXITY 
perplexity = tf.exp(loss)
with tf.Session() as session:
  print("Perplexity for RNN Trigram MOdel= "+ str(session.run(perplexity)))

  
# SAME CODE JUST CHANGED THE MODEL TO LSTM BASED
tf.reset_default_graph()
# tf Graph input
x = tf.placeholder("float", [None, n_gram, 1])
y = tf.placeholder("float", [None, vocab_size])

# RNN output node weights and biases
weights = {
    'out': tf.Variable(tf.random_normal([n_hidden, vocab_size]))
}
biases = {
    'out': tf.Variable(tf.random_normal([vocab_size]))
}

def RNN(x, weights, biases):

    x = tf.reshape(x, [-1, n_gram])

    # Generate a n_input-element sequence of inputs
    x = tf.split(x,n_gram,1)

    # 1-layer LSTM with n_hidden units 
    rnn_cell = rnn.LSTMCell(n_hidden, reuse =tf.AUTO_REUSE)

    outputs, states = rnn.static_rnn(rnn_cell, x, dtype=tf.float32)
    # there are n_input outputs but we only want the last output
    return tf.matmul(outputs[-1], weights['out']) + biases['out']

pred = RNN(x, weights, biases)

# Loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.RMSPropOptimizer(learning_rate=learning_rate).minimize(cost)

# Model evaluation
correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initializing the variables
init = tf.global_variables_initializer()

# Launch the graph
with tf.Session() as session:
    session.run(init)
    step = 0
    offset = random.randint(0,n_gram+1)
    end_offset = n_gram + 1
    acc_total = 0
    loss_total = 0
    loss_overall = 0
    writer.add_graph(session.graph)

    while step < training_iters:
        # Generate a minibatch. Add some randomness on selection process.
        if offset > (len(training_data)-end_offset):
            offset = random.randint(0, n_gram+1)

        symbols_in_keys = [ [dictionary[ str(training_data[i])]] for i in range(offset, offset+n_gram) ]
        symbols_in_keys = list(np.reshape(np.array(symbols_in_keys), [-1, n_gram, 1]))

        symbols_out_onehot = np.zeros([vocab_size], dtype=float)
        symbols_out_onehot[dictionary[str(training_data[offset+n_gram])]] = 1.0
        symbols_out_onehot = np.reshape(symbols_out_onehot,[1,-1])

        _, acc, loss, onehot_pred = session.run([optimizer, accuracy, cost, pred], \
                                                feed_dict={x: symbols_in_keys, y: symbols_out_onehot})
        loss_total += loss
        loss_overall += loss
        acc_total += acc
        if (step+1) % display_step == 0:
            print("Iter= " + str(step+1) + ", Average Loss= " + \
                  "{:.6f}".format(loss_total/display_step) + ", Average Accuracy= " + \
                  "{:.2f}%".format(100*acc_total/display_step))
            acc_total = 0
            loss_total = 0
            symbols_in = [training_data[i] for i in range(offset, offset + n_gram)]
            symbols_out = training_data[offset + n_gram]
            symbols_out_pred = reverse_dictionary[int(tf.argmax(onehot_pred, 1).eval())]
            
        step += 1
        offset += (n_gram+1)
    print("Optimization Finished!")
    print("Elapsed time: ", elapsed(time.time() - start_time))
    
    num_start = 0
    words = [word_index['<PAD>']] * (n_gram-1) + ['<s>']
    overall = list(words)
    sentence = ''
    num_words = 0
    while num_start < 5 and num_words < 500:
        try:
            symbols_in_keys = [dictionary[str(words[i])] for i in range(len(words))]
            keys = np.reshape(np.array(symbols_in_keys), [-1, n_gram, 1])
            onehot_pred = session.run(pred, feed_dict={x: keys})
            onehot_pred_index = int(tf.argmax(onehot_pred, 1).eval())
            if onehot_pred_index == dictionary['</s>']:
                sentence += ". "
                num_start += 1
            else:
                sentence = "%s %s" % (sentence,reverse_dictionary[onehot_pred_index])
            symbols_in_keys = list(symbols_in_keys[1:])
            symbols_in_keys.append(onehot_pred_index)
        except:
            onehot_pred_index = dictionary['the']
            sentence = "%s %s" % (sentence,reverse_dictionary[onehot_pred_index])
            symbols_in_keys = list(symbols_in_keys[1:])
            symbols_in_keys.append(onehot_pred_index)
        num_words += 1
        

    print(sentence)
 

#TASK_2 FINDING PERPLEXITY 
perplexity = tf.exp(loss)
with tf.Session() as session:
  print("Perplexity for LSTM Trigram MOdel= "+ str(session.run(perplexity)))

    

Iteration= 1000, Average Loss= 29.739771, Average Accuracy= 0.70%
Iteration= 2000, Average Loss= 22.695530, Average Accuracy= 2.10%
Iteration= 3000, Average Loss= 17.704835, Average Accuracy= 1.90%
Iteration= 4000, Average Loss= 14.154064, Average Accuracy= 3.50%
Iteration= 5000, Average Loss= 11.645088, Average Accuracy= 5.90%
Iteration= 6000, Average Loss= 10.247011, Average Accuracy= 5.50%
Iteration= 7000, Average Loss= 9.084210, Average Accuracy= 8.90%
Iteration= 8000, Average Loss= 8.487877, Average Accuracy= 7.30%
Iteration= 9000, Average Loss= 8.191763, Average Accuracy= 8.10%
Iteration= 10000, Average Loss= 8.260988, Average Accuracy= 7.90%
Iteration= 11000, Average Loss= 8.032297, Average Accuracy= 8.70%
Iteration= 12000, Average Loss= 8.227926, Average Accuracy= 7.00%
Iteration= 13000, Average Loss= 8.048016, Average Accuracy= 7.60%
Iteration= 14000, Average Loss= 7.970814, Average Accuracy= 7.50%
Iteration= 15000, Average Loss= 8.130755, Average Accuracy= 7.40%
Iteration= 16

In [17]:
#CODE FOR TASK-2a Vanilla RNN BASED LANGUAGE GENERATION (BIGRAMS)
tf.reset_default_graph()

# Parameters
learning_rate = 0.0001
training_iters = 40000
display_step = 1000
#CHANGE VALUE OF n_gram for getting N-GRAM MODEL
n_gram = 2


logs_path = '/tmp/tensorflow/rnn_words'
writer = tf.summary.FileWriter(logs_path)

# number of units in RNN cell
n_hidden = 128

# tf Graph input
x = tf.placeholder("float", [None, n_gram, 1])
y = tf.placeholder("float", [None, vocab_size])

# RNN output node weights and biases
weights = {
    'out': tf.Variable(tf.random_normal([n_hidden, vocab_size]))
}
biases = {
    'out': tf.Variable(tf.random_normal([vocab_size]))
}

def RNN(x, weights, biases):

    x = tf.reshape(x, [-1, n_gram])
    x = tf.split(x,n_gram,1)
    rnn_cell = rnn.BasicRNNCell(n_hidden)
    outputs, states = rnn.static_rnn(rnn_cell, x, dtype=tf.float32)
    return tf.matmul(outputs[-1], weights['out']) + biases['out']

pred = RNN(x, weights, biases)

# Loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.RMSPropOptimizer(learning_rate=learning_rate).minimize(cost)

# Model evaluation
correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initializing the variables
init = tf.global_variables_initializer()

# Launch the graph
with tf.Session() as session:
    session.run(init)
    step = 0
    offset = random.randint(0,n_gram+1)
    end_offset = n_gram + 1
    acc_total = 0
    loss_total = 0
    loss_overall = 0
    writer.add_graph(session.graph)

    while step < training_iters:
        # Generate a minibatch. Add some randomness on selection process.
        if offset > (len(training_data)-end_offset):
            offset = random.randint(0, n_gram+1)

        symbols_in_keys = [ [dictionary[ str(training_data[i])]] for i in range(offset, offset+n_gram) ]
        symbols_in_keys = list(np.reshape(np.array(symbols_in_keys), [-1, n_gram, 1]))

        symbols_out_onehot = np.zeros([vocab_size], dtype=float)
        symbols_out_onehot[dictionary[str(training_data[offset+n_gram])]] = 1.0
        symbols_out_onehot = np.reshape(symbols_out_onehot,[1,-1])

        _, acc, loss, onehot_pred = session.run([optimizer, accuracy, cost, pred], \
                                                feed_dict={x: symbols_in_keys, y: symbols_out_onehot})
        loss_total += loss
        loss_overall += loss
        acc_total += acc
        if (step+1) % display_step == 0:
            print("Iteration= " + str(step+1) + ", Average Loss= " + \
                  "{:.6f}".format(loss_total/display_step) + ", Average Accuracy= " + \
                  "{:.2f}%".format(100*acc_total/display_step))
            acc_total = 0
            loss_total = 0
            symbols_in = [training_data[i] for i in range(offset, offset + n_gram)]
            symbols_out = training_data[offset + n_gram]
            symbols_out_pred = reverse_dictionary[int(tf.argmax(onehot_pred, 1).eval())]
        step += 1
        offset += (n_gram+1)
    print("Optimization Finished!")
    print("Elapsed time: ", elapsed(time.time() - start_time))
    
    num_start = 0
    words = [word_index['<PAD>']] * (n_gram-1) + ['<s>']
    overall = list(words)
    sentence = ''
    num_words = 0
    while num_start < n_gram and num_words < 500:
        try:
            symbols_in_keys = [dictionary[str(words[i])] for i in range(len(words))]
            keys = np.reshape(np.array(symbols_in_keys), [-1, n_gram, 1])
            onehot_pred = session.run(pred, feed_dict={x: keys})
            onehot_pred_index = int(tf.argmax(onehot_pred, 1).eval())
            if onehot_pred_index == dictionary['</s>']:
                sentence += ". "
                num_start += 1
            else:
                sentence = "%s %s" % (sentence,reverse_dictionary[onehot_pred_index])
            symbols_in_keys = list(symbols_in_keys[1:])
            symbols_in_keys.append(onehot_pred_index)
        except:
            onehot_pred_index = dictionary['the']
            sentence = "%s %s" % (sentence,reverse_dictionary[onehot_pred_index])
            symbols_in_keys = list(symbols_in_keys[1:])
            symbols_in_keys.append(onehot_pred_index)
        num_words += 1
        

    print(sentence)
    

#TASK_2 FINDING PERPLEXITY 
perplexity = tf.exp(loss)
with tf.Session() as session:
  print("Perplexity for RNN Bigram MOdel= "+ str(session.run(perplexity)))

  
# SAME CODE JUST CHANGED THE MODEL TO LSTM BASED
tf.reset_default_graph()
# tf Graph input
x = tf.placeholder("float", [None, n_gram, 1])
y = tf.placeholder("float", [None, vocab_size])

# RNN output node weights and biases
weights = {
    'out': tf.Variable(tf.random_normal([n_hidden, vocab_size]))
}
biases = {
    'out': tf.Variable(tf.random_normal([vocab_size]))
}

def RNN(x, weights, biases):

    x = tf.reshape(x, [-1, n_gram])

    # Generate a n_input-element sequence of inputs
    x = tf.split(x,n_gram,1)

    # 1-layer LSTM with n_hidden units 
    rnn_cell = rnn.LSTMCell(n_hidden, reuse =tf.AUTO_REUSE)

    outputs, states = rnn.static_rnn(rnn_cell, x, dtype=tf.float32)
    # there are n_input outputs but we only want the last output
    return tf.matmul(outputs[-1], weights['out']) + biases['out']

pred = RNN(x, weights, biases)

# Loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.RMSPropOptimizer(learning_rate=learning_rate).minimize(cost)

# Model evaluation
correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initializing the variables
init = tf.global_variables_initializer()

# Launch the graph
with tf.Session() as session:
    session.run(init)
    step = 0
    offset = random.randint(0,n_gram+1)
    end_offset = n_gram + 1
    acc_total = 0
    loss_total = 0
    loss_overall = 0
    writer.add_graph(session.graph)

    while step < training_iters:
        # Generate a minibatch. Add some randomness on selection process.
        if offset > (len(training_data)-end_offset):
            offset = random.randint(0, n_gram+1)

        symbols_in_keys = [ [dictionary[ str(training_data[i])]] for i in range(offset, offset+n_gram) ]
        symbols_in_keys = list(np.reshape(np.array(symbols_in_keys), [-1, n_gram, 1]))

        symbols_out_onehot = np.zeros([vocab_size], dtype=float)
        symbols_out_onehot[dictionary[str(training_data[offset+n_gram])]] = 1.0
        symbols_out_onehot = np.reshape(symbols_out_onehot,[1,-1])

        _, acc, loss, onehot_pred = session.run([optimizer, accuracy, cost, pred], \
                                                feed_dict={x: symbols_in_keys, y: symbols_out_onehot})
        loss_total += loss
        loss_overall += loss
        acc_total += acc
        if (step+1) % display_step == 0:
            print("Iter= " + str(step+1) + ", Average Loss= " + \
                  "{:.6f}".format(loss_total/display_step) + ", Average Accuracy= " + \
                  "{:.2f}%".format(100*acc_total/display_step))
            acc_total = 0
            loss_total = 0
            symbols_in = [training_data[i] for i in range(offset, offset + n_gram)]
            symbols_out = training_data[offset + n_gram]
            symbols_out_pred = reverse_dictionary[int(tf.argmax(onehot_pred, 1).eval())]
            
        step += 1
        offset += (n_gram+1)
    print("Optimization Finished!")
    print("Elapsed time: ", elapsed(time.time() - start_time))
    
    num_start = 0
    words = [word_index['<PAD>']] * (n_gram-1) + ['<s>']
    overall = list(words)
    sentence = ''
    num_words = 0
    while num_start < 5 and num_words < 500:
        try:
            symbols_in_keys = [dictionary[str(words[i])] for i in range(len(words))]
            keys = np.reshape(np.array(symbols_in_keys), [-1, n_gram, 1])
            onehot_pred = session.run(pred, feed_dict={x: keys})
            onehot_pred_index = int(tf.argmax(onehot_pred, 1).eval())
            if onehot_pred_index == dictionary['</s>']:
                sentence += ". "
                num_start += 1
            else:
                sentence = "%s %s" % (sentence,reverse_dictionary[onehot_pred_index])
            symbols_in_keys = list(symbols_in_keys[1:])
            symbols_in_keys.append(onehot_pred_index)
        except:
            onehot_pred_index = dictionary['the']
            sentence = "%s %s" % (sentence,reverse_dictionary[onehot_pred_index])
            symbols_in_keys = list(symbols_in_keys[1:])
            symbols_in_keys.append(onehot_pred_index)
        num_words += 1
        

    print(sentence)
 

#TASK_2 FINDING PERPLEXITY 
perplexity = tf.exp(loss)
with tf.Session() as session:
  print("Perplexity for LSTM Bigram MOdel= "+ str(session.run(perplexity)))

    

Iteration= 1000, Average Loss= 29.840953, Average Accuracy= 0.80%
Iteration= 2000, Average Loss= 22.384513, Average Accuracy= 2.70%
Iteration= 3000, Average Loss= 18.773488, Average Accuracy= 0.50%
Iteration= 4000, Average Loss= 14.604409, Average Accuracy= 2.70%
Iteration= 5000, Average Loss= 11.732413, Average Accuracy= 5.70%
Iteration= 6000, Average Loss= 10.430374, Average Accuracy= 8.60%
Iteration= 7000, Average Loss= 9.042752, Average Accuracy= 7.80%
Iteration= 8000, Average Loss= 8.614702, Average Accuracy= 6.90%
Iteration= 9000, Average Loss= 8.236876, Average Accuracy= 7.60%
Iteration= 10000, Average Loss= 8.230106, Average Accuracy= 8.10%
Iteration= 11000, Average Loss= 8.014003, Average Accuracy= 8.20%
Iteration= 12000, Average Loss= 7.891100, Average Accuracy= 8.40%
Iteration= 13000, Average Loss= 8.137641, Average Accuracy= 7.90%
Iteration= 14000, Average Loss= 7.885893, Average Accuracy= 7.40%
Iteration= 15000, Average Loss= 8.070992, Average Accuracy= 7.00%
Iteration= 16

In [18]:
#CODE FOR TASK-2a Vanilla RNN BASED LANGUAGE GENERATION (UNIGRAMS)
tf.reset_default_graph()

# Parameters
learning_rate = 0.0001
training_iters = 40000
display_step = 1000
#CHANGE VALUE OF n_gram for getting N-GRAM MODEL
n_gram = 1


logs_path = '/tmp/tensorflow/rnn_words'
writer = tf.summary.FileWriter(logs_path)

# number of units in RNN cell
n_hidden = 128

# tf Graph input
x = tf.placeholder("float", [None, n_gram, 1])
y = tf.placeholder("float", [None, vocab_size])

# RNN output node weights and biases
weights = {
    'out': tf.Variable(tf.random_normal([n_hidden, vocab_size]))
}
biases = {
    'out': tf.Variable(tf.random_normal([vocab_size]))
}

def RNN(x, weights, biases):

    x = tf.reshape(x, [-1, n_gram])
    x = tf.split(x,n_gram,1)
    rnn_cell = rnn.BasicRNNCell(n_hidden)
    outputs, states = rnn.static_rnn(rnn_cell, x, dtype=tf.float32)
    return tf.matmul(outputs[-1], weights['out']) + biases['out']

pred = RNN(x, weights, biases)

# Loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.RMSPropOptimizer(learning_rate=learning_rate).minimize(cost)

# Model evaluation
correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initializing the variables
init = tf.global_variables_initializer()

# Launch the graph
with tf.Session() as session:
    session.run(init)
    step = 0
    offset = random.randint(0,n_gram+1)
    end_offset = n_gram + 1
    acc_total = 0
    loss_total = 0
    loss_overall = 0
    writer.add_graph(session.graph)

    while step < training_iters:
        # Generate a minibatch. Add some randomness on selection process.
        if offset > (len(training_data)-end_offset):
            offset = random.randint(0, n_gram+1)

        symbols_in_keys = [ [dictionary[ str(training_data[i])]] for i in range(offset, offset+n_gram) ]
        symbols_in_keys = list(np.reshape(np.array(symbols_in_keys), [-1, n_gram, 1]))

        symbols_out_onehot = np.zeros([vocab_size], dtype=float)
        symbols_out_onehot[dictionary[str(training_data[offset+n_gram])]] = 1.0
        symbols_out_onehot = np.reshape(symbols_out_onehot,[1,-1])

        _, acc, loss, onehot_pred = session.run([optimizer, accuracy, cost, pred], \
                                                feed_dict={x: symbols_in_keys, y: symbols_out_onehot})
        loss_total += loss
        loss_overall += loss
        acc_total += acc
        if (step+1) % display_step == 0:
            print("Iteration= " + str(step+1) + ", Average Loss= " + \
                  "{:.6f}".format(loss_total/display_step) + ", Average Accuracy= " + \
                  "{:.2f}%".format(100*acc_total/display_step))
            acc_total = 0
            loss_total = 0
            symbols_in = [training_data[i] for i in range(offset, offset + n_gram)]
            symbols_out = training_data[offset + n_gram]
            symbols_out_pred = reverse_dictionary[int(tf.argmax(onehot_pred, 1).eval())]
        step += 1
        offset += (n_gram+1)
    print("Optimization Finished!")
    print("Elapsed time: ", elapsed(time.time() - start_time))
    
    num_start = 0
    words = [word_index['<PAD>']] * (n_gram-1) + ['<s>']
    overall = list(words)
    sentence = ''
    num_words = 0
    while num_start < n_gram and num_words < 500:
        try:
            symbols_in_keys = [dictionary[str(words[i])] for i in range(len(words))]
            keys = np.reshape(np.array(symbols_in_keys), [-1, n_gram, 1])
            onehot_pred = session.run(pred, feed_dict={x: keys})
            onehot_pred_index = int(tf.argmax(onehot_pred, 1).eval())
            if onehot_pred_index == dictionary['</s>']:
                sentence += ". "
                num_start += 1
            else:
                sentence = "%s %s" % (sentence,reverse_dictionary[onehot_pred_index])
            symbols_in_keys = list(symbols_in_keys[1:])
            symbols_in_keys.append(onehot_pred_index)
        except:
            onehot_pred_index = dictionary['the']
            sentence = "%s %s" % (sentence,reverse_dictionary[onehot_pred_index])
            symbols_in_keys = list(symbols_in_keys[1:])
            symbols_in_keys.append(onehot_pred_index)
        num_words += 1
        

    print(sentence)
    

#TASK_2 FINDING PERPLEXITY 
perplexity = tf.exp(loss)
with tf.Session() as session:
  print("Perplexity for RNN Unigram MOdel= "+ str(session.run(perplexity)))

  
# SAME CODE JUST CHANGED THE MODEL TO LSTM BASED
tf.reset_default_graph()
# tf Graph input
x = tf.placeholder("float", [None, n_gram, 1])
y = tf.placeholder("float", [None, vocab_size])

# RNN output node weights and biases
weights = {
    'out': tf.Variable(tf.random_normal([n_hidden, vocab_size]))
}
biases = {
    'out': tf.Variable(tf.random_normal([vocab_size]))
}

def RNN(x, weights, biases):

    x = tf.reshape(x, [-1, n_gram])

    # Generate a n_input-element sequence of inputs
    x = tf.split(x,n_gram,1)

    # 1-layer LSTM with n_hidden units 
    rnn_cell = rnn.LSTMCell(n_hidden, reuse =tf.AUTO_REUSE)

    outputs, states = rnn.static_rnn(rnn_cell, x, dtype=tf.float32)
    # there are n_input outputs but we only want the last output
    return tf.matmul(outputs[-1], weights['out']) + biases['out']

pred = RNN(x, weights, biases)

# Loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.RMSPropOptimizer(learning_rate=learning_rate).minimize(cost)

# Model evaluation
correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initializing the variables
init = tf.global_variables_initializer()

# Launch the graph
with tf.Session() as session:
    session.run(init)
    step = 0
    offset = random.randint(0,n_gram+1)
    end_offset = n_gram + 1
    acc_total = 0
    loss_total = 0
    loss_overall = 0
    writer.add_graph(session.graph)

    while step < training_iters:
        # Generate a minibatch. Add some randomness on selection process.
        if offset > (len(training_data)-end_offset):
            offset = random.randint(0, n_gram+1)

        symbols_in_keys = [ [dictionary[ str(training_data[i])]] for i in range(offset, offset+n_gram) ]
        symbols_in_keys = list(np.reshape(np.array(symbols_in_keys), [-1, n_gram, 1]))

        symbols_out_onehot = np.zeros([vocab_size], dtype=float)
        symbols_out_onehot[dictionary[str(training_data[offset+n_gram])]] = 1.0
        symbols_out_onehot = np.reshape(symbols_out_onehot,[1,-1])

        _, acc, loss, onehot_pred = session.run([optimizer, accuracy, cost, pred], \
                                                feed_dict={x: symbols_in_keys, y: symbols_out_onehot})
        loss_total += loss
        loss_overall += loss
        acc_total += acc
        if (step+1) % display_step == 0:
            print("Iter= " + str(step+1) + ", Average Loss= " + \
                  "{:.6f}".format(loss_total/display_step) + ", Average Accuracy= " + \
                  "{:.2f}%".format(100*acc_total/display_step))
            acc_total = 0
            loss_total = 0
            symbols_in = [training_data[i] for i in range(offset, offset + n_gram)]
            symbols_out = training_data[offset + n_gram]
            symbols_out_pred = reverse_dictionary[int(tf.argmax(onehot_pred, 1).eval())]
            
        step += 1
        offset += (n_gram+1)
    print("Optimization Finished!")
    print("Elapsed time: ", elapsed(time.time() - start_time))
    
    num_start = 0
    words = [word_index['<PAD>']] * (n_gram-1) + ['<s>']
    overall = list(words)
    sentence = ''
    num_words = 0
    while num_start < 5 and num_words < 500:
        try:
            symbols_in_keys = [dictionary[str(words[i])] for i in range(len(words))]
            keys = np.reshape(np.array(symbols_in_keys), [-1, n_gram, 1])
            onehot_pred = session.run(pred, feed_dict={x: keys})
            onehot_pred_index = int(tf.argmax(onehot_pred, 1).eval())
            if onehot_pred_index == dictionary['</s>']:
                sentence += ". "
                num_start += 1
            else:
                sentence = "%s %s" % (sentence,reverse_dictionary[onehot_pred_index])
            symbols_in_keys = list(symbols_in_keys[1:])
            symbols_in_keys.append(onehot_pred_index)
        except:
            onehot_pred_index = dictionary['the']
            sentence = "%s %s" % (sentence,reverse_dictionary[onehot_pred_index])
            symbols_in_keys = list(symbols_in_keys[1:])
            symbols_in_keys.append(onehot_pred_index)
        num_words += 1
        

    print(sentence)
 

#TASK_2 FINDING PERPLEXITY 
perplexity = tf.exp(loss)
with tf.Session() as session:
  print("Perplexity for LSTM Unigram MOdel= "+ str(session.run(perplexity)))

    

Iteration= 1000, Average Loss= 27.839959, Average Accuracy= 0.00%
Iteration= 2000, Average Loss= 20.673776, Average Accuracy= 0.00%
Iteration= 3000, Average Loss= 15.664599, Average Accuracy= 0.40%
Iteration= 4000, Average Loss= 12.179997, Average Accuracy= 1.60%
Iteration= 5000, Average Loss= 9.889768, Average Accuracy= 3.30%
Iteration= 6000, Average Loss= 8.713016, Average Accuracy= 6.90%
Iteration= 7000, Average Loss= 8.471187, Average Accuracy= 7.60%
Iteration= 8000, Average Loss= 8.307948, Average Accuracy= 8.30%
Iteration= 9000, Average Loss= 8.324764, Average Accuracy= 8.10%
Iteration= 10000, Average Loss= 8.129380, Average Accuracy= 8.30%
Iteration= 11000, Average Loss= 7.950933, Average Accuracy= 8.10%
Iteration= 12000, Average Loss= 8.023893, Average Accuracy= 7.40%
Iteration= 13000, Average Loss= 7.958294, Average Accuracy= 8.10%
Iteration= 14000, Average Loss= 8.081017, Average Accuracy= 8.20%
Iteration= 15000, Average Loss= 8.218112, Average Accuracy= 6.80%
Iteration= 1600

In [22]:
print("Overehere the Neural network displayed comparatively poor results with respect to classical method. \nHowever, fine tuning of parameters will result in better results by the Neural netwrok,\n as they get trained with the context and learn the features directly from the text. Morover, the ")

Overehere the Neural network displayed comparatively poor results with respect to classical method. 
However, fine tuning of parameters will result in better results by the Neural netwrok,
 as they get trained with the context and learn the features directly from the text. Morover, the 
